# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123127e+02     1.711691e+00
 * time: 0.09424495697021484
     1     1.047991e+01     9.343160e-01
 * time: 0.9960160255432129
     2    -1.193138e+01     1.098977e+00
 * time: 1.1534209251403809
     3    -3.420650e+01     8.748666e-01
 * time: 1.2942039966583252
     4    -4.756674e+01     6.124611e-01
 * time: 1.454693078994751
     5    -5.706021e+01     2.250515e-01
 * time: 1.6062099933624268
     6    -5.976222e+01     2.140850e-01
 * time: 1.714108943939209
     7    -6.086317e+01     8.495412e-02
 * time: 1.8326799869537354
     8    -6.134776e+01     4.194778e-02
 * time: 1.947390079498291
     9    -6.165709e+01     3.384703e-02
 * time: 2.081897020339966
    10    -6.184861e+01     3.518770e-02
 * time: 2.190023899078369
    11    -6.198747e+01     2.957360e-02
 * time: 2.3081259727478027
    12    -6.206315e+01     2.920424e-02
 * time: 2.4161579608917236
    13    -6.211824e+01     1.666897e-02
 * time: 2.53065299987792

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671059
    AtomicLocal         -18.8557787
    AtomicNonlocal      14.8522758
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485379 
    Xc                  -19.3336821

    total               -62.261666454496
